# API Lime API

In [12]:
import requests
import numpy as np
import pandas as pd

from datetime import datetime
import pytz

## Date and time processing

In [23]:
def get_current_date_time():
    # Get the current time in the UTC timezone
    utc_now = datetime.now(pytz.utc)

    # Convert the UTC time to the Paris timezone
    paris_tz = pytz.timezone('Europe/Paris')
    paris_now = utc_now.astimezone(paris_tz)

    return paris_now

## Get dataframe from lime's API

In [36]:
url = "https://velib-metropole-opendata.smoove.pro/opendata/Velib_Metropole/station_status.json"

try:
    response = requests.get(url)
    response.raise_for_status()  # Check for any errors in the response

    # If the request was successful, parse the JSON data
    data = response.json()

    # Extract the 'stations' list from the JSON data
    stations_data = data["data"]["stations"]

    # Create a DataFrame from the 'stations' list
    df = pd.DataFrame(stations_data)

    # Get the current date and time
    current_date_time = get_current_date_time()

    #Convert into year, month, day, hour and minute 
    df["year"] = current_date_time.year
    df["month"] = current_date_time.month
    df["day"] = current_date_time.day
    df["hour"] = current_date_time.hour
    df["minute"] = current_date_time.minute
    
except requests.exceptions.RequestException as e:
    print("Error fetching data:", e)

df.head()

,stationCode,station_id,num_bikes_available,numBikesAvailable,num_bikes_available_types,num_docks_available,numDocksAvailable,is_installed,is_returning,is_renting,last_reported,year,month,day,hour,minute
0,16107,213688169,6,6,"[{'mechanical': 3}, {'ebike': 3}]",29,29,1,1,1,1690463591,2023,7,27,15,27
1,31104,653222953,23,23,"[{'mechanical': 12}, {'ebike': 11}]",6,6,1,1,1,1690463574,2023,7,27,15,27
2,9020,36255,8,8,"[{'mechanical': 1}, {'ebike': 7}]",13,13,1,1,1,1690463552,2023,7,27,15,27
3,12109,37815204,4,4,"[{'mechanical': 2}, {'ebike': 2}]",21,21,1,1,1,1690463668,2023,7,27,15,27
4,14111,251039991,10,10,"[{'mechanical': 6}, {'ebike': 4}]",15,15,1,1,1,1690463626,2023,7,27,15,27


In [27]:
df.shape

(1454, 16)

## Data cleaning

Dropping every station that is not installed.

In [37]:
df = df[df['is_installed'] != 0]
df.shape

(1454, 16)

Get the interesting columns.

In [39]:
df = df[["stationCode","num_bikes_available","numDocksAvailable","year","month","day","hour","minute"]]
df.head()

,stationCode,num_bikes_available,numDocksAvailable,year,month,day,hour,minute
0,16107,6,29,2023,7,27,15,27
1,31104,23,6,2023,7,27,15,27
2,9020,8,13,2023,7,27,15,27
3,12109,4,21,2023,7,27,15,27
4,14111,10,15,2023,7,27,15,27
